In [121]:
import numpy as np
import pandas as pd
import glob

import matplotlib
import matplotlib.pyplot as plt

from tqdm import tqdm
import numba

import bokeh
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [122]:
#load each run into a separate dataframe
df1 = pd.read_csv('..\\190926_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df2 = pd.read_csv('..\\191007_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df3 = pd.read_csv('..\\191010_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df4 = pd.read_csv('..\\191015_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df5 = pd.read_csv('..\\191020_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df6 = pd.read_csv('..\\191021_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df7 = pd.read_csv('..\\191022_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df8 = pd.read_csv('..\\191025_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df9 = pd.read_csv('..\\191027_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df10 = pd.read_csv('..\\191031_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df11 = pd.read_csv('..\\191106_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df12 = pd.read_csv('..\\191115_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df13 = pd.read_csv('..\\191119_Deuterium_Transfer_Peak_Areas.csv', comment='#')

In [123]:
#add a column to each dataframe listing the date
df1['GCMS Date'] = '190926'
df2['GCMS Date'] = '191007'
df3['GCMS Date'] = '191010'
df4['GCMS Date'] = '191015'
df5['GCMS Date'] = '191020'
df6['GCMS Date'] = '191021'
df7['GCMS Date'] = '191022'
df8['GCMS Date'] = '191025'
df9['GCMS Date'] = '191027'
df10['GCMS Date'] = '191031'
df11['GCMS Date'] = '191106'
df12['GCMS Date'] = '191115'
df13['GCMS Date'] = '191119'

#merge data frames into one dataframe
outlier_df=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10).append(df11).append(df12).append(df13)
outlier_df = outlier_df[['WellNumber','Outlier','GCMS Date']].drop_duplicates().astype(str)
outlier_df.head()

,WellNumber,Outlier,GCMS Date
0,1,N,190926
6,2,N,190926
12,3,BI,190926
18,4,BI,190926
24,5,N,190926


In [124]:
#Glob together all csvs with the deeplabcut generated position data. 
#The deeplabcut data lists the network iteration of the trained network at the end of the filename
#This value will always be a multiple of 1000
file_list=glob.glob('./*/*000.csv')
well_list=glob.glob('./*/*Well_Positions.csv')

#Date sync contains the dates that the video and GCMS data were collected on
#These two dates are different for every trial
date_sync = pd.read_csv('..\\DCHC_Date_sync.csv')

In [125]:
file_list[:10]

['.\\190919\\190919_DCHC_transfer_experiment_S_well_1DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_2DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_3DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_4DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_5DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_6DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_7DeepCut_resnet50_DCHC_transfer_analysis_20190919_SOct2shuffle1_80000.csv',
 '.\\190919\\190919_DCHC_transfer_experiment_S_well_8DeepCut_resnet50_DCHC_transfer_analysis_20190919_SO

In [126]:
well_list

['.\\Well_positions\\190919_DCHC_transfer_experiment_S_Well_Positions.csv',
 '.\\Well_positions\\191003_DCHC_transfer_experiment_SP_Well_Positions.csv',
 '.\\Well_positions\\191008_DCHC_transfer_experiment_P_Well_Positions.csv',
 '.\\Well_positions\\191012_DCHC_transfer_experiment_SP_Well_Positions.csv',
 '.\\Well_positions\\191016_DCHC_transfer_experiment_P_Well_Positions.csv',
 '.\\Well_positions\\191019_DCHC_transfer_experiment_S_Well_Positions.csv',
 '.\\Well_positions\\191020_DCHC_transfer_experiment_S_Well_Positions.csv',
 '.\\Well_positions\\191021_DCHC_transfer_experiment_SP_Well_Positions.csv',
 '.\\Well_positions\\191022_DCHC_transfer_experiment_P_Well_Positions.csv',
 '.\\Well_positions\\191025_DCHC_transfer_experiment_S_Well_Positions.csv',
 '.\\Well_positions\\191029_DCHC_transfer_experiment_D_Well_Positions.csv',
 '.\\Well_positions\\191105_DCHC_transfer_experiment_D_Well_Positions.csv',
 '.\\Well_positions\\191113_DCHC_transfer_experiment_P_Well_Positions.csv',
 '.\\Well

In [127]:
for i,name in tqdm(enumerate(well_list)):
    #load each csv as a dataframe and append to the end of one large dataframe
    if i == 0:
        well_df=pd.read_csv(name,comment='#')
    else:
        well_df=well_df.append(pd.read_csv(name,comment='#'))

14it [00:00, 147.82it/s]


In [128]:
well_df

,,X,Y,WellNum,Date
0,1,533,271,1,190919
1,2,368,551,1,190919
2,3,215,275,1,190919
3,4,530,272,2,190919
4,5,367,551,2,190919
...,...,...,...,...,...
19,20,377,577,7,191115
20,21,211,290,7,191115
21,22,535,290,8,191115
22,23,368,573,8,191115


In [129]:
#create dataframe for metadata\
#data index keeps track of index in df_list so that any reordering of 
#metadata during datafram merges doesn't mess up indexing 
metadata=pd.DataFrame(columns=['WellNumber','species','Video Date','file path','data index'])
#list to contain all position dataframes
df_list={}
for i,name in tqdm(enumerate(file_list)):
    #load each csv as a dataframe into a list
    df_list.update({str(i) : pd.read_csv(name,header=[1,2])})
    
    #pull metadata out of the filenames (Species identifier, collection date, well number)
    metadata = metadata.append({'species':name[name.rfind('_',0,name.rfind('_'))+1], 
                        'Video Date':name[name.find('\\')+1:name.find('\\')+7],
                        'WellNumber':name[name.find('well')+5:name.find('well')+6],
                        'file path' : name,
                        'data index' : i},
                     ignore_index=True)

#creating columns for the ant length information
metadata['length lower']=''
metadata['length upper']=''

109it [00:15,  7.18it/s]


In [130]:
#loop through the each dataframe and create a new column named antlength
#antlength is the length of the ant calculated from the head and gaster position data
for i,df in tqdm(enumerate(df_list.items())):
    df['antlength'] = np.sqrt((df['anthead']['x'] - df['antgaster']['x'])**2 + 
                           (df['anthead']['y'] - df['antgaster']['y'])**2)

0it [00:00, ?it/s]


TypeError: tuple indices must be integers or slices, not str

In [ ]:
#list all dates in the metadata file
np.unique(metadata['Video Date'])

### In the next section the ant lengths from each experiment are plotted in order to determine an upper and lower bound for the length of the ant in each well

In [ ]:
#count is used to specify the position of each subplot
count=0

#date specifies the date of the data collection to be analyzed
date='191105'

#loop through the indices of the rows from the ID dataframe. These indices should match
#up with the position of the corresponding dataframe in the dataframe list.
for i in metadata.loc[metadata['Video Date']==date].index:
    df=df_list[str(i)]
    subplt = int(str(len(metadata.loc[metadata['Video Date']==date].index)) + '1' + str(count+1))
    plt.figure(figsize=(10,20))
    plt.subplot(subplt)
    plt.title('Well ' + metadata.iloc[i]['WellNumber'])
    plt.scatter(df['bodyparts']['coords'][::1000],df['antlength'][::1000],alpha=0.1)
    plt.ylim([40,120])
    count = count+1

# Thought
Maybe don't need to have a lower bound. Isn't too big of an issue if either head or gaster is incorrectly labelled so long as it is closer to the other marker than what it should be

In [ ]:
#metadata['length lower'].loc[metadata['Video Date']=='190919'] = 40
metadata['length upper'].loc[metadata['Video Date']=='190919'] = 120
#metadata['length lower'].loc[metadata['Video Date']=='191003'] = 40
metadata['length upper'].loc[metadata['Video Date']=='191003'] = 120
#metadata['length lower'].loc[metadata['Video Date']=='191008'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191008'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191012'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191012'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191016'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191016'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191019'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191019'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191020'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191020'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191021'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191021'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191022'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191022'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191025'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191025'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191029'] = 45
metadata['length upper'].loc[metadata['Video Date']=='191029'] = 125
#metadata['length lower'].loc[metadata['Video Date']=='191105'] = 40
metadata['length upper'].loc[metadata['Video Date']=='191105'] = 120
#metadata['length lower'].loc[metadata['Video Date']=='191113'] = 30
metadata['length upper'].loc[metadata['Video Date']=='191113'] = 110
#metadata['length lower'].loc[metadata['Video Date']=='191115'] = 40
metadata['length upper'].loc[metadata['Video Date']=='191115'] = 120

In [112]:
metadata.head()

,WellNumber,species,Video Date,file path,data index,length lower,length upper
0,1,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,0,,120
1,2,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,1,,120
2,3,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,2,,120
3,4,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,3,,120
4,5,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,4,,120


In [113]:
date_sync.head()

,Video Date,GCMS Date,WellNumber,Outlier
0,190919,190926,1,N
1,190919,190926,2,N
2,190919,190926,3,BI
3,190919,190926,4,BI
4,190919,190926,5,N


In [114]:
outlier_df

,WellNumber,Outlier,GCMS Date
0,1,N,190926
6,2,N,190926
12,3,BI,190926
18,4,BI,190926
24,5,N,190926
...,...,...,...
18,4,N,191119
24,5,N,191119
30,6,N,191119
36,7,N,191119


In [115]:
date_sync = date_sync.astype(str).merge(outlier_df)
metadata = metadata.merge(date_sync)

In [116]:
metadata.head()

,WellNumber,species,Video Date,file path,data index,length lower,length upper,GCMS Date,Outlier
0,1,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,0,,120,190926,N
1,2,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,1,,120,190926,N
2,3,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,2,,120,190926,BI
3,4,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,3,,120,190926,BI
4,5,S,190919,.\190919\190919_DCHC_transfer_experiment_S_wel...,4,,120,190926,N


In [117]:
metadata=metadata.loc[(metadata['Outlier']!='Y') | (metadata['Outlier']!='BI')]

In [118]:
test_dict={'1':'Red','2':'Blue','3':'Green'}

In [120]:
test_dict['1']

'Red'

In [67]:
df_list = [df_list[i] for i in metadata['data index']]

In [50]:
for i,df in enumerate(df_list):
    #df['good frames'] = (df['antlength'] < metadata.iloc[i]['length upper']) & (df['antlength'] > metadata.iloc[i]['length lower'])  
    df['good frames'] = (df['antlength'] < metadata['length upper'].loc[metadata['data index'] == i].values[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [66]:
metadata['data index'].values

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108],
      dtype=object)

In [69]:
df_list

[      bodyparts     anthead                          antgaster              \
          coords           x           y likelihood           x           y   
 0             0  427.767942  399.401206        1.0  361.601315  394.884507   
 1             1  428.026720  399.485341        1.0  361.411461  395.220237   
 2             2  428.056251  399.431414        1.0  361.357584  395.020876   
 3             3  427.648703  399.881099        1.0  361.437400  394.975922   
 4             4  427.595800  399.978169        1.0  361.298920  395.039374   
 ...         ...         ...         ...        ...         ...         ...   
 89655     89655  476.983001  166.695163        1.0  541.239096  172.728185   
 89656     89656  476.977633  166.803819        1.0  541.259258  172.825200   
 89657     89657  476.969556  166.753766        1.0  541.259038  172.731162   
 89658     89658  476.942515  166.746576        1.0  541.300613  172.719304   
 89659     89659  476.935356  166.732293        1.0 

In [57]:
i

29

In [19]:
for i,df in enumerate(df_list):
    avgLen = np.average(df['antlength'].loc[df['good frames']])
    df['touching'] = False

    df['head2head'] = np.sqrt((df['beetlehead']['x'] - df['anthead']['x'])**2 + 
                              (df['beetlehead']['y'] - df['anthead']['y'])**2)
        
    df['head2butt'] = np.sqrt((df['beetlehead']['x'] - df['antgaster']['x'])**2 + 
                              (df['beetlehead']['y'] - df['antgaster']['y'])**2)
    
    if (metadata.iloc[i]['species'] == 'S') | (metadata.iloc[i]['species'] == 'D'):
        df['ellipse distance'] = df['head2head'] + df['head2butt']
        df['touching'] = ((df['antlength']*1.4 > df['ellipse distance'])&df['good frames']) | ((df['head2head']<avgLen) | (df['head2butt']<avgLen))
        
    if metadata.iloc[i]['species'] == 'P':
        df['butt2head'] = np.sqrt((df['beetleabdomen']['x'] - df['anthead']['x'])**2 + 
                              (df['beetleabdomen']['y'] - df['anthead']['y'])**2)
        
        df['butt2butt'] = np.sqrt((df['beetleabdomen']['x'] - df['antgaster']['x'])**2 + 
                              (df['beetleabdomen']['y'] - df['antgaster']['y'])**2)
    
        headEllipse=df['head2head'] + df['head2butt']
        buttEllipse=df['butt2head'] + df['butt2butt']
    
        df['ellipse distance'] = headEllipse*(headEllipse<buttEllipse) + buttEllipse*(buttEllipse<headEllipse)
    
        df['touching'] = ((df['antlength']*1.4 > df['ellipse distance'])&df['good frames']) \
                            | ((df['head2head']<avgLen) | (df['head2butt']<avgLen))\
                            | ((df['butt2head']<avgLen) | (df['butt2butt']<avgLen))

KeyError: 'beetleabdomen'

In [ ]:
sum(df_list[1]['touching'])

In [ ]:
metadata['touching percent']=''
for i,df in enumerate(df_list):
    metadata['touching percent'].iloc[i] = (sum(df['touching'])/len(df))

In [ ]:
plt.scatter(metadata['species'],metadata['touching percent'],alpha=0.7)

In [ ]:
metadata.head()

In [ ]:
metadata.to_csv(path_or_buf='.\Movement_metadata.csv')

In [ ]:
well_df.head()

In [ ]:
for i,df in enumerate(df_list):
    positions=well_df.loc[(well_df['Date'] == int(metadata['Video Date'].iloc[i]))&(well_df['WellNum'] == int(metadata['WellNumber'].iloc[i]))]
    positions = positions.append(positions.iloc[2]*0,ignore_index=True)
    #determine the distance between a label and the three antechamber entrance locations
    df['ant dist1']=np.sqrt((df['anthead']['x']-positions.iloc[0]['X'])**2 + (df['anthead']['y']-positions.iloc[0]['Y'])**2)
    df['ant dist2']=np.sqrt((df['anthead']['x']-positions.iloc[1]['X'])**2 + (df['anthead']['y']-positions.iloc[1]['Y'])**2)
    df['ant dist3']=np.sqrt((df['anthead']['x']-positions.iloc[2]['X'])**2 + (df['anthead']['y']-positions.iloc[2]['Y'])**2)

    df['beetle dist1']=np.sqrt((df['beetlehead']['x']-positions.iloc[0]['X'])**2 + (df['beetlehead']['y']-positions.iloc[0]['Y'])**2)
    df['beetle dist2']=np.sqrt((df['beetlehead']['x']-positions.iloc[1]['X'])**2 + (df['beetlehead']['y']-positions.iloc[1]['Y'])**2)
    df['beetle dist3']=np.sqrt((df['beetlehead']['x']-positions.iloc[2]['X'])**2 + (df['beetlehead']['y']-positions.iloc[2]['Y'])**2)

    #determine which of the three antechamber entrances is closest to the given point
    df['ant closest']=0*((df['ant dist1']<df['ant dist2'])&(df['ant dist1']<df['ant dist3'])) + \
                       1*((df['ant dist2']<df['ant dist1'])&(df['ant dist2']<df['ant dist3'])) + \
                       2*((df['ant dist3']<df['ant dist1'])&(df['ant dist3']<df['ant dist2']))

    df['beetle closest']=0*((df['beetle dist1']<df['beetle dist2'])&(df['beetle dist1']<df['beetle dist3'])) + \
                       1*((df['beetle dist2']<df['beetle dist1'])&(df['beetle dist2']<df['beetle dist3'])) + \
                       2*((df['beetle dist3']<df['beetle dist1'])&(df['beetle dist3']<df['beetle dist2']))

    #determine the equation of the circle circumscribing the central well of the arena using the three points marking the entrances to the antechamber
    #basing this off of Zaz's comment from https://math.stackexchange.com/questions/213658/get-the-equation-of-a-circle-when-given-3-points
    x=positions['X'][:3].values
    y=positions['Y'][:3].values
    ones=np.ones(3)
    xy=(positions['X'][:3]**2 + positions['Y'][:3]**2).values

    #creating the minors ofthe matrix of the system of equations
    M11=np.stack((x,y,ones),axis=1)
    M12=np.stack((xy,y,ones),axis=1)
    M13=np.stack((xy,x,ones),axis=1)
    M14=np.stack((xy,x,y),axis=1)

    #calculating the center of the circle as well as the radius
    x_0=1/2 * np.linalg.det(M12)/np.linalg.det(M11)
    y_0=-1/2 * np.linalg.det(M13)/np.linalg.det(M11)
    r=np.sqrt(x_0**2+y_0**2+(np.linalg.det(M14)/np.linalg.det(M11)))

    #determine if the ant and the beetle are inside or outside of the circle
    #plug in the position of the beetle into the left side of (x_0-x)**2 + (y_0-y)**2 < r**2
    #and determine if the value is smaller than than the right side
    df['ant in circle'] = np.sqrt((df['anthead']['x']-x_0)**2 + (df['anthead']['y']-y_0)**2) < r
    df['beetle in circle'] = np.sqrt((df['beetlehead']['x']-x_0)**2 + (df['beetlehead']['y']-y_0)**2) < r

    #creating columns for the location of the beetle and ant. If the beetle is in the center of the circle,
    #the location coding is 3, otherwise it is 0-3 starting in the bottom right chamber working counterclockwise
    df['ant loc'] = 3*df['ant in circle'] + (df['ant closest'] & ~df['ant in circle'])
    df['beetle loc'] = 3*df['beetle in circle'] + (df['beetle closest'] & ~df['beetle in circle'])
    
    #determining all of the possible distances that need to be measured
    #euclidean distance between the ant and the beetle
    euclidean_dist=df['head2head']

    #distance from the beetle to the nearest antechamber entrance
    beetle2beetleentrance=np.sqrt((df['beetlehead']['x']-positions['X'].iloc[df['beetle closest']].values)**2 + \
                                  (df['beetlehead']['y']-positions['Y'].iloc[df['beetle closest']].values)**2)

    #distance from the beetle to the antechamber entrance nearest the ant
    beetle2antentrance=np.sqrt((df['beetlehead']['x']-positions['X'].iloc[df['ant closest']].values)**2 + \
                               (df['beetlehead']['y']-positions['Y'].iloc[df['ant closest']].values)**2)

    #distance from the ant to the nearest antechamber entrance
    ant2antentrance=np.sqrt((df['anthead']['x']-positions['X'].iloc[df['ant closest']].values)**2 + \
                            (df['anthead']['y']-positions['Y'].iloc[df['ant closest']].values)**2)

    #distance from the ant to the antechamber entrance nearest the beetle
    ant2beetleentrance=np.sqrt((df['anthead']['x']-positions['X'].iloc[df['beetle closest']].values)**2 + \
                               (df['anthead']['y']-positions['Y'].iloc[df['beetle closest']].values)**2)

    #distance between the two antechamber entrances
    entrance2entrance=np.sqrt((positions['X'].iloc[df['ant closest']].values-positions['X'].iloc[df['beetle closest']].values)**2 + \
                              (positions['Y'].iloc[df['ant closest']].values-positions['Y'].iloc[df['beetle closest']].values)**2)
    
    df['actual distance'] = (df['ant loc']==df['beetle loc'])*euclidean_dist + \
                             (df['ant loc']!=df['beetle loc'])* \
                             ((df['ant loc']==3)*(ant2beetleentrance+beetle2beetleentrance) + \
                              (df['beetle loc']==3)*(beetle2antentrance+ant2antentrance) + \
                              ((df['ant loc']!=3)*(df['beetle loc']!=3))* \
                               (ant2antentrance+beetle2beetleentrance+entrance2entrance))

In [ ]:
df_list[0].head()

In [ ]:
def ecdf_func(x):
    return np.sort(x), np.arange(len(x))/len(x)

In [ ]:
colors=['Red','Orange','Green','Blue','Purple']
plt.figure(figsize=(20,20))
for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        color='red'
    if metadata['species'].iloc[i] == 'S':
        color='blue'
    if metadata['species'].iloc[i] == 'D':
        color='purple'
        
    x_val,y_val=ecdf_func(df['head2head'][::100])
    
    plt.scatter(x_val,y_val,color=color,alpha=0.2)

In [ ]:
colors=['Red','Orange','Green','Blue','Purple']
plt.figure(figsize=(20,20))
for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        color='red'
    if metadata['species'].iloc[i] == 'S':
        color='blue'
    if metadata['species'].iloc[i] == 'D':
        color='purple'
        
    x_val,y_val=ecdf_func(df['actual distance'][::100])
    
    plt.scatter(x_val,y_val,color=color,alpha=0.2)

In [ ]:
platy_set=[]
scepto_set=[]
dalotia_set=[]
for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        platy_set = np.append(platy_set,df['actual distance'].values)
    if metadata['species'].iloc[i] == 'S':
        scepto_set = np.append(scepto_set,df['actual distance'].values)
    if metadata['species'].iloc[i] == 'D':
        dalotia_set = np.append(dalotia_set,df['actual distance'].values)

In [ ]:
len(dalotia_set)

In [ ]:
thinning=1000
plt.figure(figsize=(20,20))
x_platy,y_platy=ecdf_func(platy_set[::thinning])
x_scepto,y_scepto=ecdf_func(scepto_set[::thinning])
x_dalotia,y_dalotia=ecdf_func(dalotia_set[::thinning])
plt.scatter(x_platy,y_platy,color='red',alpha=0.2)
plt.scatter(x_scepto,y_scepto,color='blue',alpha=0.2)
plt.scatter(x_dalotia,y_dalotia,color='purple',alpha=0.2)

In [ ]:
@numba.njit
def draw_bs_sample(data,size=1000):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=size)

In [ ]:
plt.figure(figsize=(20,20))
for _ in range(300):    
    x_platy,y_platy=ecdf_func(draw_bs_sample(platy_set))
    x_scepto,y_scepto=ecdf_func(draw_bs_sample(scepto_set))
    x_dalotia,y_dalotia=ecdf_func(draw_bs_sample(dalotia_set))
    plt.scatter(x_platy,y_platy,color='red',alpha=0.01)
    plt.scatter(x_scepto,y_scepto,color='blue',alpha=0.01)
    plt.scatter(x_dalotia,y_dalotia,color='purple',alpha=0.01)
    
x_platy,y_platy=ecdf_func(platy_set[::thinning])
x_scepto,y_scepto=ecdf_func(scepto_set[::thinning])
x_dalotia,y_dalotia=ecdf_func(dalotia_set[::thinning])
plt.plot(x_platy,y_platy,color='black')
plt.plot(x_scepto,y_scepto,color='black')
plt.plot(x_dalotia,y_dalotia,color='black')

In [ ]:
colors=['Red','Orange','Green','Blue','Purple']
plt.figure(figsize=(20,20))
for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        color='red'
    if metadata['species'].iloc[i] == 'S':
        color='blue'
    if metadata['species'].iloc[i] == 'D':
        color='purple'
        
    x_val,y_val=ecdf_func(df['actual distance'][::100])
    
    plt.scatter(x_val,y_val,color=color,alpha=0.2)
    
x_platy,y_platy=ecdf_func(platy_set[::thinning])
x_scepto,y_scepto=ecdf_func(scepto_set[::thinning])
x_dalotia,y_dalotia=ecdf_func(dalotia_set[::thinning])
plt.plot(x_platy,y_platy,color='black',linewidth=10)
plt.plot(x_scepto,y_scepto,color='black',linewidth=10)
plt.plot(x_dalotia,y_dalotia,color='black',linewidth=10)

In [ ]:
p = bokeh.plotting.figure(plot_height = 600, plot_width = 600, 
            x_axis_label = 'Distance, pixels', 
            y_axis_label = 'Frames at Given Distance')

for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        color='#C42F2F'
    if metadata['species'].iloc[i] == 'S':
        color='#5F56FF'
    if metadata['species'].iloc[i] == 'D':
        color='#832161'
    
    arr_hist, edges = np.histogram(df['actual distance'][::10],800,range = [0, 800])
        
    # Add a quad glyph
    p.quad(bottom=0, top=arr_hist, 
           left=edges[0:-1], right=edges[1:], 
           fill_color=color, line_color=None,alpha=0.7)

bokeh.io.show(p)

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(30,20))

for i,df in enumerate(df_list): 
    if metadata['species'].iloc[i] == 'P':
        color='#C42F2F'
    if metadata['species'].iloc[i] == 'S':
        color='#5F56FF'
    if metadata['species'].iloc[i] == 'D':
        color='#832161'

    sns.distplot(df['actual distance'][::10], hist = False, kde = True,
                 kde_kws = {'linewidth': 5,'color':color,'alpha':0.6})


# Plot formatting
plt.xlabel('Distance')
plt.ylabel('Density')